# 第9章 数据聚合与分组运算

在将数据集处理好了以后，通常的任务就是分组统计或者生成数据透视表。pandas提供了一个很好的功能。

# 主要内容——1.GroupBy  
1.1 基本方法  
1.2 将分组进行迭代  
1.3 选取一个或一组列进行分组  
1.4 通过字典或者Series进行分组  
1.5 通过函数进行分组  
1.6 根据索引级别分组

# 1.1 基本方法

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.721424,0.784564,a,one
1,1.597930,0.166909,a,two
2,0.390308,-2.179971,b,one
3,0.597726,0.407132,b,two
4,-0.830468,0.901558,a,one


In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
grouped.mean()

key1
a   -0.317987
b    0.494017
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -1.275946
      two     1.597930
b     one     0.390308
      two     0.597726
Name: data1, dtype: float64

这里介绍stack和unstack方法。   
常见的数据的层次化结构有两种，一种是表格，一种是“花括号”，即下面这样的l两种形式


In [14]:
from IPython.display import Image
Image(url = '1.png', width = 250, height = 250)

In [17]:
Image(url = '2.png', width = 300, height = 300)

表格在行列方向上均有索引（类似于DataFrame），花括号结构只有“列方向”上的索引（类似于层次化的Series），结构更加偏向于堆叠（Series-stack，方便记忆）。stack函数会将数据从”表格结构“变成”花括号结构“，即将其行索引变成列索引，反之，unstack函数将数据从”花括号结构“变成”表格结构“，即要将其中一层的列索引变成行索引

In [8]:
means.unstack()

key2,one,two
key1,,
a,-1.275946,1.597930
b,0.390308,0.597726


分组键可以是任意长度适当的数组

In [18]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.597930
            2006    0.390308
Ohio        2005   -0.561849
            2006   -0.830468
Name: data1, dtype: float64

也可以将列名当作分组键（字符串、数字或者其他）

In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.317987,0.617677
b,0.494017,-0.886420


In [20]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.275946  0.843061
     two   1.597930  0.166909
b    one   0.390308 -2.179971
     two   0.597726  0.407132

In [21]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

**注意：** df.groupby('key1').mean()结果中没有key2列。这是由于df['key2']不是数值数据，因此在结果中默认被排除了。

# 1.2 将分组进行迭代

GroupBy支持迭代，可以产生一组二元元祖。

In [22]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.721424  0.784564    a  one
1  1.597930  0.166909    a  two
4 -0.830468  0.901558    a  one
b
      data1     data2 key1 key2
2  0.390308 -2.179971    b  one
3  0.597726  0.407132    b  two


在多重键的情况下，元祖的第一个元素将会是由键组成的元祖

In [26]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -1.721424  0.784564    a  one
4 -0.830468  0.901558    a  one
('a', 'two')
     data1     data2 key1 key2
1  1.59793  0.166909    a  two
('b', 'one')
      data1     data2 key1 key2
2  0.390308 -2.179971    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.597726  0.407132    b  two


也可以将这些数据片段做成一个字典

In [27]:
list(df.groupby('key1'))

[('a',       data1     data2 key1 key2
  0 -1.721424  0.784564    a  one
  1  1.597930  0.166909    a  two
  4 -0.830468  0.901558    a  one), ('b',       data1     data2 key1 key2
  2  0.390308 -2.179971    b  one
  3  0.597726  0.407132    b  two)]

In [28]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,0.390308,-2.179971,b,one
3,0.597726,0.407132,b,two


In [29]:
pieces['a']

,data1,data2,key1,key2
0,-1.721424,0.784564,a,one
1,1.597930,0.166909,a,two
4,-0.830468,0.901558,a,one


groupby默认是在axis=0上进行分组，也可以设置在其他轴上进行分组。比如，可以根据dtype对列进行分组。

In [31]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [33]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -1.721424  0.784564
 1  1.597930  0.166909
 2  0.390308 -2.179971
 3  0.597726  0.407132
 4 -0.830468  0.901558, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

# 1.3 选取一个或一组列进行分组

In [35]:
df.groupby('key1')['data1'].mean()

key1
a   -0.317987
b    0.494017
Name: data1, dtype: float64

In [36]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.843061
     two   0.166909
b    one  -2.179971
     two   0.407132

# 1.4 通过字典或者Series进行分组

除字典外，分组信息还可以以其他形式存在，比如：

In [38]:
people = pd.DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.770779,-0.394608,0.898982,-0.264950,0.604316
Steve,-1.178254,0.208871,0.408423,-1.107895,0.512845
Wes,1.025344,NaN,NaN,0.398280,-0.607738
Jim,-0.202656,0.854771,-1.253251,0.290644,1.284759
Travis,1.150472,-0.753442,0.586572,1.046285,-0.127861


假设已知列的分组关系，我们希望根据分组来计算列的总计

In [39]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [40]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.634032,-0.561071
Steve,-0.699471,-0.456538
Wes,0.398280,0.417606
Jim,-0.962607,1.936874
Travis,1.632857,0.269169


Series也具有同样的功能，它可以被看作是一个固定大小的映射

In [42]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [43]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


# 1.5 通过函数进行分组

相比于字典和Series，根据函数进行分组更加地抽象。以上一节的people数据集为例，其索引为人名。假如我们希望根据人名的长度进行分组，那么需要传入len函数。

In [44]:
people

,a,b,c,d,e
Joe,-0.770779,-0.394608,0.898982,-0.264950,0.604316
Steve,-1.178254,0.208871,0.408423,-1.107895,0.512845
Wes,1.025344,NaN,NaN,0.398280,-0.607738
Jim,-0.202656,0.854771,-1.253251,0.290644,1.284759
Travis,1.150472,-0.753442,0.586572,1.046285,-0.127861


In [45]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.051909,0.460162,-0.354269,0.423974,1.281338
5,-1.178254,0.208871,0.408423,-1.107895,0.512845
6,1.150472,-0.753442,0.586572,1.046285,-0.127861


也可以先转换一个变量，再进行分组

In [52]:
people['len'] = people.index.map(len)

In [56]:
people.groupby('len').sum()

,a,b,c,d,e
len,,,,,
3,0.051909,0.460162,-0.354269,0.423974,1.281338
5,-1.178254,0.208871,0.408423,-1.107895,0.512845
6,1.150472,-0.753442,0.586572,1.046285,-0.127861


也可以将函数跟数组、字典、列表、Series混合使用

In [57]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e  len
3 one -0.770779 -0.394608  0.898982 -0.264950 -0.607738    3
  two -0.202656  0.854771 -1.253251  0.290644  1.284759    3
5 one -1.178254  0.208871  0.408423 -1.107895  0.512845    5
6 two  1.150472 -0.753442  0.586572  1.046285 -0.127861    6

# 1.6 根据索引级别分组

多重索引时，根据level关键字传入级别参数进行索引

In [59]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.279661  0.650815 -1.417361  0.547038  1.461937
1      0.547552  0.799081 -2.637028  0.592365  0.335392
2     -1.283605 -1.271535  0.228263 -0.164727 -0.016074
3      0.509149  0.752425  0.364218 -1.689831  0.052757

In [61]:
hier_df.groupby(level='cty', axis=1).sum()

cty,JP,US
0,2.008975,-0.486885
1,0.927757,-1.290395
2,-0.180801,-2.326878
3,-1.637074,1.625792
